In [18]:
import os
from pathlib import Path
from zipfile import ZipFile as zipfile
import sys
import time
import glob
import scipy
import numpy as np
import pandas as pd
import torch
import argparse
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torch.autograd import Variable
import shutil

In [19]:
#from google.colab import drive
#drive.mount('/content/drive')

In [22]:
image_path = Path("/content/drive/MyDrive/Colab Notebooks")
input_folder = image_path / "dataset"
if not os.path.exists(input_folder):
  with zipfile(image_path/"dataset.zip", 'r') as zipf:
      print("Extracting files...")
      zipf.extractall(image_path)

Extracting files...


In [23]:
device = torch.device('cuda:0')
batch_size = 64
torch.backends.cudnn.benchmark = True

In [24]:
data = pd.read_csv(image_path / "train.csv")

data_pic = data["image"]
data_y = data["Class"]

In [25]:
total_classes = len(data_y.unique())

In [26]:
pic_train, pic_test, y_train, y_test = train_test_split(data_pic, data_y, train_size=0.8, shuffle=True)

In [27]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

train_transform = transforms.Compose([
          transforms.Resize((224,224)),
          transforms.RandomResizedCrop(224),
          transforms.RandomHorizontalFlip(),
          transforms.ColorJitter(
            brightness=0.4,
            contrast=0.4,
            saturation=0.4,
            hue=0.2),
          transforms.ToTensor(),
          normalize,
        ])

test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
  ])

In [28]:
Path(image_path / "dataset" / "train" ).mkdir(exist_ok = True, parents = True)
Path(image_path / "dataset" / "test" ).mkdir(exist_ok = True, parents = True)

In [29]:
if not os.path.exists(image_path / "dataset" / "train" / "13"):
  for image_name in pic_train:
    try:
      Path(image_path / "dataset" / "train" / str(data[data["image"] == image_name]["Class"].values[0])).mkdir(exist_ok = True, parents = True)
      shutil.move(
          input_folder / image_name, 
          input_folder / "train" / str(data[data["image"] == image_name]["Class"].values[0]) / image_name
          )
    except:
      pass
  for image_name in pic_test:
    try:
      Path(image_path / "dataset" / "test" / str(data[data["image"] == image_name]["Class"].values[0])).mkdir(exist_ok = True, parents = True)
      shutil.move(
          input_folder / image_name, 
          input_folder / "test" / str(data[data["image"] == image_name]["Class"].values[0]) / image_name
          )
    except:
      pass

In [30]:
trainset = torchvision.datasets.ImageFolder(os.path.join(input_folder, "train"), transform=train_transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, num_workers=2, shuffle=True, pin_memory = True)

testset = torchvision.datasets.ImageFolder(os.path.join(input_folder, "test"), transform=test_transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, num_workers=2, shuffle=True, pin_memory = True)

In [36]:
from torchvision.models import ResNet50_Weights

model_ft = models.resnet50(weights=ResNet50_Weights.DEFAULT)
num_ftrs = model_ft.fc.in_features

model_ft.fc = nn.Linear(num_ftrs, 196)
model_ft = model_ft.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_ft.parameters(), lr=0.01, momentum=0.9)

lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold = 0.9)

In [37]:
def accuracy(output, target):
    with torch.no_grad():
        pred = torch.argmax(output, dim=1)
        assert pred.shape[0] == len(target)
        correct = 0
        correct += torch.sum(pred == target).item()
    return correct / len(target)

In [41]:
def train_model(model, criterion, optimizer, scheduler, n_epochs = 5):
    
    losses = []
    accuracies = []
    test_accuracies = []
    model.train()
    for epoch in range(n_epochs):
        since = time.time()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):

            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()

        epoch_duration = time.time()-since
        epoch_loss = running_loss/len(train_loader)
        print("Epoch %s, duration: %d s, loss: %.4f" % (epoch+1, epoch_duration, epoch_loss))
        
        losses.append(epoch_loss)
        
        model.eval()
        test_acc = eval_model(model)
        test_accuracies.append(test_acc)
        
        model.train()
        scheduler.step(test_acc)
        since = time.time()
    print('Finished Training')
    return model, losses, test_accuracies


In [42]:
def eval_model(model):
    correct = 0.0
    total = 0.0
    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model_ft(images)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc = 100.0 * correct / total
    print('test acc: %d %%' % (
        test_acc))
    return test_acc

In [ ]:
model_ft, training_losses, test_accs = train_model(model_ft, criterion, optimizer, lrscheduler, n_epochs=10)

Epoch 1, duration: 125 s, loss: 4.6913
ttest acc: 12 %
Epoch 2, duration: 137 s, loss: 3.5376
ttest acc: 32 %
Epoch 3, duration: 125 s, loss: 2.4854
ttest acc: 47 %
Epoch 4, duration: 121 s, loss: 1.8490
ttest acc: 59 %
Epoch 5, duration: 121 s, loss: 1.4370
ttest acc: 65 %
Epoch 6, duration: 123 s, loss: 1.2289
ttest acc: 70 %


In [17]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
model_saved = torch.jit.script(model_ft)
model_saved.save("model_resnet50.pt")